In [1]:
import pandas as pd
from pykml.factory import KML_ElementMaker as KML
from lxml import etree

In [2]:
filename = 'processos_fisicos.xlsx'

In [3]:
df = pd.read_excel(io=filename)

In [4]:
df.head()

,Processo,Tipo,Nome,Latitude,Longitude,Obs
0,44860500030,CGH,Usina do Poço,-28.1308,-51.6281,2016
1,23390567107,CGH,Toca,-29.2885,-50.7406,2016
2,30730500057,CGH,Taipinha,-28.9231,-52.5067,2016
3,120750500035,CGH,Soledade,-28.9372,-52.4846,2016
4,108000500056,CGH,N/D,-29.6892,-55.4108,2016


In [5]:
doc = KML.Document()

In [6]:
icons = {
    'branco':'http://maps.google.com/mapfiles/kml/pushpin/wht-pushpin.png',
}

s = KML.Style(
        KML.IconStyle(
            KML.Icon(
                KML.href(icons['branco'])
            ),
        ),
        id='estilo',
    )

doc.append(s)

In [7]:
import re
def conversion(coord):
    deg, minutes, seconds, direction =  re.split('[°\'"]', coord)
    return (float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction in ['W', 'S'] else 1)

In [8]:
for index, row in df.iterrows():
    name = row['Tipo']+' '+row['Nome']
    processo = row['Processo']
    tipo = row['Tipo']
    description = '''
Processo: {}
Tipo: {}
    '''.format(processo, tipo)
    lat = row['Latitude']
    long = row['Longitude']
    if isinstance(lat, str):
        lat = conversion(lat)
        long = conversion(long)
    coordinates = str(long)+','+str(lat)
        
    p = KML.Placemark(
        KML.name(name),
        KML.Point(KML.coordinates(coordinates)),
        KML.description(description),
        KML.styleUrl('#estilo'))
     
    doc.append(p)

In [9]:
kml_file_path = 'hidreletricas_fisicos.kml'

In [10]:
kml_str = etree.tostring(doc, pretty_print=True).decode('utf-8')

In [11]:
f = open(kml_file_path, "w")
f.write(kml_str)
f.close()